We consider a two-dimensional truss structures, consisting of $m$ bars connecting $n$ nodes.
An external force $F$ is applied to each nodes of the truss, and cause a small displacement $X$ of the nodes. 

The goal of this exercise is to optimize the width $w_i$'s (or rather, the cross section) of each bar ($i=1,\ldots,m$),
subject to a constraint on the total volume of the structure, in order to minimize the compliance of the truss. The compliance
is defined as $C(\mathbf{w}) = \langle X, F \rangle$, and can be interpreted as the amount of elastic energy stored in the truss.

We represent $F$ and $X$ by $n \times 2$ matrices, 
where $F_{id}$ (or $X_{id}$) represents the value
of the force (displacement) at node $i$ along dimension $d$. In other words,
the $i$th row of $F$ (or $X$) is the two-dimensional vector of the external force (displacement) at the $i$th node.
The displacement of the truss structure is prescribed to be $0$ on some nodes in certain dimensions (because of mechanical links). We denote by 

$$\mathcal{S} = \{(i,d): \text{node $i$ may move along direction $d$}\} \subseteq 
[n]\times[2]$$

the set of *degrees of freedom* (DoF) of the truss, and we let $k=|\mathcal{S}|$ denote the number of DoFs. Then, 
we represent by $\mathbf{f}=\{F_{id}\}_{(i,d)\in\mathcal{S}}$ and 
$\mathbf{x}=\{X_{id}\}_{(i,d)\in\mathcal{S}}$ the k-dimensional
vectors of force and displacement along the DoFs.

The physics tells us that the vector of displacements satisifies
$$K(\mathbf{w})\ \mathbf{x} = \mathbf{f},$$
where $K(\mathbf{w}) = \sum_{i=1}^m w_i \mathbf{v}_i \mathbf{v}_i^T\in\mathbb{S}_+^k$, and the vectors $\mathbf{v}_i \in \mathbb{R}^{k}$ depend only on the geometry of the truss.
The quantity to minimize is hence
$$
C(\mathbf{w}) = \langle \mathbf{x}, \mathbf{f} \rangle
= \mathbf{f}^T K(\mathbf{w})^{-1} \mathbf{f}
$$

We consider the case of a single constraint on the total volume of the truss,
$\sum_{i} w_i L_i \leq W$,
where $L_i$ denotes the length of the $i$th bar.

So, The problem to solve is
$$
\underset{\textbf{w}\in\mathbb{R}_+^m}{minimize}\  \Big\{\ \mathbf{f}^T K(\mathbf{w})^{-1} \mathbf{f}\qquad s.t. \quad
\sum_{i} w_i L_i \leq W \Big\}
$$

#### Import some packages

In [ ]:
import numpy as np
import cvxopt as cvx
import matplotlib
import matplotlib.pyplot as plt
import picos
get_ipython().magic(u'matplotlib inline')
matplotlib.rcParams['figure.figsize'] = (4, 4)

Below, we define a function that 
returns the variables `nodes` (coordinate of truss nodes),`bars` (list of pairs of nodes), and `dofs` ({0,1}-array of the same size as `nodes`, which indicates the degrees of freedom of the truss).

In [ ]:
def small_structure():
    nodes = np.array([
    [0,1],#node 0
    [1,1],#node 1
    [0,0],#node 2
    [1,0],#node 3
    [2,0],#node 4
    ])

    dofs = np.array([
        [0,0],#node 0 does not move
        [1,1],#node 1 is free
        [0,1],#node 2 can move vertically
        [1,1],#node 3 is free
        [1,1],#node 4 is free
        ])

    bars = [[0,1],
            [0,2],
            [1,2],
            [2,3],
            [1,3],
            [1,4],
            [3,4]]
    return nodes,dofs,bars


Now, we define some physical constants, as well as two functions that
return the required constant vectors/matrices required to compute the stiffness matrix $K(\mathbf{w})$.

The function `stiffness_vectors` returns the list of vectors $\boldsymbol{v}_i$'s such that the stiffness matrix is equal to $\sum_i w_i \boldsymbol{v}_i \boldsymbol{v}_i^ T$, where $w_i$ is the scaling factor of bar $i$.

The function `stiffness_matrix(w)` returns the matrix $K(\mathbf{w}):=\sum_i w_i \boldsymbol{v}_i \boldsymbol{v}_i^ T$.

#### define some constants

In [ ]:
g=9.8     #gravity on earth
A0 = 0.01 #unit section of a bar (in m2)
D = 450.  #density of the bar (kg/m3)
E = 9 #young elasticity modulus (in GPa) --> so u = K^-1 f is in nm

In [ ]:
def stiffness_vectors():
    v = []
    n = len(nodes)
    m = len(bars)
    d = nodes.shape[1]
    dof_mask = np.concatenate(dofs,axis=0)
    for k,(i,j) in enumerate(bars):
        L = np.linalg.norm(nodes[i]-nodes[j])
        angles = (nodes[j]-nodes[i])/L
        vk = np.array([0.]*d*n)
        vk[d*i:d*(i+1)] = angles
        vk[d*j:d*(j+1)] = -angles
        vk *= (E*A0/L)**0.5
        v.append(cvx.matrix(vk[dof_mask==1]))
    return v
            
def stiffness_matrix(w=None):
    n = len(bars)
    if w is None:
        w=[1]*n
    v = stiffness_vectors()
    m = len(v[0])
    K = cvx.matrix(0,(m,m))
    for i in range(n):
        K = K + w[i]*v[i]*v[i].T
    return K

This function draws the truss (defined in global vars `nodes`,`dofs`
and `bars`), with optional bar widths `w` (we assume unit widths if `w` is not provided), and if a force vector `f` is provided, the displacement of the structure, amplified by a factor `delta`.

In [ ]:
def draw_truss(w=None,f=None, delta = 1.):
    if w is None:
        w=[1]*len(bars)
    for k,(i,j) in enumerate(bars):
        plt.plot([nodes[i][0],nodes[j][0]],[nodes[i][1],nodes[j][1]],linewidth = 3*w[k],color='blue')
    
    minx,miny = np.min(nodes,axis=0)
    maxx,maxy = np.max(nodes,axis=0)
    plt.xlim(minx-0.2,maxx+0.2)
    plt.ylim(miny-0.2,maxy+.2)
    
    if f is not None:
        K = stiffness_matrix(w=w)
        dof_mask = np.concatenate(dofs,axis=0)
        ff = f.ravel()[dof_mask==1] #projection on coordinates of the degrees of freedom
        dx = np.zeros(nodes.shape[0]*nodes.shape[1])
        dx[dof_mask==1] = np.linalg.lstsq(K,ff,rcond=-1)[0]*1e-9 #1e-9 because E is in GPa
        dx= np.reshape(dx,nodes.shape)
        dnodes = nodes + delta * dx
        for k,(i,j) in enumerate(bars):
            plt.plot([dnodes[i][0],dnodes[j][0]],[dnodes[i][1],dnodes[j][1]],linewidth = 3*w[k],color='red')

OK, now, let's draw the truss that we defined in the function `small_structure()`.

In [ ]:
nodes,dofs,bars = small_structure()
draw_truss()

In the next cell, we draw this truss again (in blue), together with the displacements (in red) caused by a vertical force applied to the 3 bottom nodes:

In [ ]:
#we define a force f that puts 500 kg on the 3 nodes below.
#note that according to `nodes` and `dofs`, this corresponds to the
#3rd, 5th, and 7th degrees of freedom.
f = np.array([
     [0,0],
     [0,0],
     [0,-g*500],# 500kg vertically on node 2
     [0,-g*500],# 500kg vertically on node 3
     [0,-g*500],# 500kg vertically on node 4
     ])
draw_truss(f=f,delta=50)

To work with a more interesting example, we provide a function which creates the
structure of a crane.
The function takes as arguments the total height and width of the crane,
the height `cr` of the top part of the crane, and the
maximum length of considered cross bars (1,2,or 3).

The function further returns a force vector $F$ (in fact, a $n \times 2$ matrix), which corresponds to the situation where a force
of coordinates (`ff[0],ff[1]`) is applied to the extremity of the crane.

In [ ]:
def crane_structure(cr=2,height=8,width=8,max_len=2,ff=(-500,-1000)):
    nodes = []
    for i in range(width):
        for j in range(height):
            if j>=height-cr-1 or i==2 or i==3:
                nodes.append((i,j))

    nodes = np.array(nodes) 

    dofs = np.ones(nodes.shape)
    dofs[:,0][nodes[:,1]==0]=0
    dofs[:,1][nodes[:,1]==0]=0

    n = len(nodes)
    bars = []
    for i in range(n):
        for j in range(n):
            if i<j and np.linalg.norm(nodes[i]-nodes[j])<1.5:
                bars.append([i,j])
            if max_len>=2 and i<j and np.linalg.norm(nodes[i]-nodes[j])==5**0.5:
                bars.append([i,j])
            if max_len>=3 and i<j and np.linalg.norm(nodes[i]-nodes[j])==10**0.5:
                bars.append([i,j])
                
    F = np.zeros((n,2))
    F[n-1-cr,0]=g*ff[0]
    F[n-1-cr,1]=g*ff[1]
    
    return nodes,dofs,bars,F

We are going to solve the optimization problem defined at the beginning of this notebook, for $W=50$.
The length $L_i$ of the bars is defined in the list `Lbars` below.
Let us first have a look at the crane structure, 
and the displacement for a "uniform" solution, where 
$w_i=\frac{W}{\sum_{j=1}^m L_j}$ for all bars.

In [ ]:
nodes,dofs,bars,F = crane_structure(cr=2,height=10,width=10,max_len=3,ff=(-500,-1000))
Lbars = [np.linalg.norm(nodes[i]-nodes[j]) for i,j in bars]
n,m = len(nodes),len(bars)
dof_mask = np.concatenate(dofs,axis=0)
f = F.ravel()[dof_mask==1]   #this projects the force on the coordinates of DoF
W = 50.

w = [1./sum(Lbars)*W]*m
draw_truss(w=w,f=F)

<span style="color:blue">
In the cell below, you should formulate the optimization problem (minimize the 
compliance $\boldsymbol{f}^T K(\mathbf{w})^{-1} \boldsymbol{f}$ of the crane,
subject to the volume constraint $\sum_{i} w_i L_i \leq W$), as an SDP.
</span>

<span style="color:blue">
*Hint:* to add a block-LMI in PICOS, of the form 
$$
\left(\begin{array}{cc}
A & B\\
B^T & C
\end{array}\right) \succeq 0,
$$
you can use the following command:

``P.add_constraint( ( (A & B) // (B.T & C) ) >> 0)``
</span>

In [ ]:
n = len(nodes)
m = len(bars)
LL = picos.new_param('LL',Lbars)
f = picos.new_param('f',f/1e6) #divide by 1e6 for the sake of numerical accuracy
W = picos.new_param('W',50.)

#create the problem
P = picos.Problem()

#add the variable for the bar widths
w = P.add_variable('w',m)
#create the stiffness matrix (NB, this is an affine expression w.r.t. `w`)
K = stiffness_matrix(w)


#constraints on the bar sizes:
P.add_constraint(w>=0)
P.add_constraint( (w|LL) <= W )


#TODO: add other variables, constraints, and set objective...

#solve the problem and display the solution
#(you can set verbose=True to see the full output of the solver)
sol = P.solve(verbose=False)
draw_truss(w=w.value,f=F)

<span style="color:blue">
Now, formulate as an SDP the problem of minimizing the worst case compliance  $f^TK^{−1}f$, over the set of all force vectors $f$ such that $\Vert f \Vert_2 \leq 1$.
That is:
$$
\underset{\textbf{w}\in\mathbb{R}_+^m}{minimize}\  \Big\{\ \sup_{\mathbf{f}: \|\mathbf{f}\|_2\leq 1} \mathbf{f}^T K(\mathbf{w})^{-1} \mathbf{f}\ |\quad
\sum_{i} w_i L_i \leq W \Big\}
$$
</span>

<span style="color:blue">
*Hint:* You should first reformulate the objective function of the problem as something for which you know a semidefinite representation...
</span>

*TODO* Write the semidefinite representation of the objective function in this cell:

(...)
    
So, $$\sup_{\mathbf{f}: \|\mathbf{f}\|_2\leq 1} \mathbf{f}^T K(\mathbf{w})^{-1} \mathbf{f} \leq t \iff (...)$$

In [ ]:
#Hint: You will need the identity matrix of size k:
I = picos.new_param('I',np.eye(K.size[0]))

#create the problem
P = picos.Problem()


#add the variable for the bar widths
w = P.add_variable('w',m)
#create the stiffness matrix (NB, this is an affine expression w.r.t. `w`)
K = stiffness_matrix(w)

#constraints on the bar sizes:
P.add_constraint(w>=0)
P.add_constraint( (w|LL) <= W )


#TODO add other variables, constraints, and objective...


#solve the problem (you can set verbose=True to see the full output of the solver)
sol = P.solve(verbose=False)
#and display the solution
draw_truss(w=w.value)

<span style="color:blue">
A somewhat more realistic load model is that of a force f acting in an unknown direction, but only at the extremity of the crane. More precisely, let $U$ be the $k\times 2$
whose columns are orthonormal and span the subspace of the 2 DoFs of the node at the extremity of the crane. Then, we want to solve
$$
\underset{\textbf{w}\in\mathbb{R}_+^m}{minimize}\  \Big\{\ \sup_{\mathbf{f}\in\operatorname{Im}(U): \|\mathbf{f}\|_2\leq 1} \mathbf{f}^T K(\mathbf{w})^{-1} \mathbf{f}\ |\quad
\sum_{i} w_i L_i \leq W \Big\}.
$$
Formulate this problem as an SDP and solve implement it in the next cell.
</span>

<span style="color:blue">
*Hint:* You should first try to find a semidefinite representation of the objective function.
</span>


*TODO* Write the semidefinite representation of the objective function in this cell:

(...)
    
So, $$\sup_{\mathbf{f}\in\operatorname{Im}(U): \|\mathbf{f}\|_2\leq 1} \mathbf{f}^T K(\mathbf{w})^{-1} \mathbf{f}\ \leq t \iff (...)$$

In [ ]:
#This are the indices of the degrees of freedom at the extremity of the crane
i0 = np.where(np.array(f.value)!=0)[0]
assert(len(i0)==2) # we check that there are just 2 DoFs

# We construct the matrix U
k = f.size[0]
U = np.zeros((k,2))
U[i0[0],0] = 1
U[i0[1],1] = 1

#You might also need the identity matrix of size 2:
I2 = picos.new_param('I2',np.eye(2))
U = picos.new_param('U',U)


#create the problem
P = picos.Problem()


#add the variable for the bar widths
w = P.add_variable('w',m)
#create the stiffness matrix (NB, this is an affine expression w.r.t. `w`)
K = stiffness_matrix(w)

#constraints on the bar sizes:
P.add_constraint(w>=0)
P.add_constraint( (w|LL) <= W )


#TODO add other variables, constraints, and objective...

#Solve the problem and display the solution 
#(leave the option solve_via_dual=False, otherwise you will run into numerical issues...)
sol = P.solve(verbose=True,solve_via_dual=False)
draw_truss(w=w.value)